<a href="https://colab.research.google.com/github/angiellanos/Merton_con_procesos_difusi-n_con_saltos/blob/main/Modelos_de_Merton_con_saltos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODELOS DE MERTON CON PROCESOS DE DIFUSIÓN CON SALTOS**
## **Universidad Nacional de Colombia**
# Derivados de Renta Variable
---
Elaborado por:
###Angie Eloisa Llanos
###Brayan Arturo Camargo
###Camilo Andres Figueroa
---



# 1. **Introducción**